In [1]:
import pickle
import pandas as pd

f = open("result_converged", "rb")
res = pickle.load(f)
f.close()

telesat_passes_df = pd.read_pickle('../Telesat_0_7day.pkl')

N_passes = 20
telesat_passes_df = telesat_passes_df[0:N_passes]

In [2]:
from matplotlib.lines import Line2D 
import numpy as np
import matplotlib.pyplot as plt
from PlotUtil import apply_report_formatting, apply_report_formatting_single
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "xelatex",
    'font.family': 'sans-serif',
    'text.usetex': False,
    'pgf.rcfonts': False
})

Ptx_dBm = np.arange(10, 40, 5)
isend_req = 10.69

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)
x = res.F[:,1]
y = ((res.F[:,0] * -1) / 1e9)   # Gigabit

cmap = plt.get_cmap('tab10')
cmap = np.array(cmap.colors)

c = cmap[res.X[:,-1].tolist(),:]
ax.scatter(x, y, c=c, marker='.', s=1)

ids = range(res.F.shape[0])
ids = [2, 3, 4, 5]
for i in ids:
    x = res.F[i, 1]
    y = (res.F[i, 0] * -1) / 1e9 
    ax.scatter(x, y, c='k', marker='.', s=0.01)
    ax.text(x - 200 ,y+ 0.1, "%d" % i, fontsize=6)
    
leg_lines = [Line2D([0], [0], color=cmap[i, :], lw=1) for i in np.unique(res.X[:,-1])]
leg_label = ["%.0f dBm" % Ptx_dBm[i] for i in np.unique(res.X[:,-1])]

ax.legend(leg_lines, leg_label, loc=4)

ax.set_xlabel("Energy used [J]")
ax.set_ylabel("Throughput [Gb]")
apply_report_formatting()
plt.tight_layout()
plt.show()

plt.savefig('D:/git/thesis_report_ae/figures/link_budget_pareto_numbered.pgf')
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_pareto_numbered.svg')
plt.close()

IndexError: index 40 is out of bounds for axis 0 with size 10

In [ ]:
# Plot comparison
fig = plt.figure(figsize=(3.2, 2.4))
ax = fig.add_subplot(1,1,1)

passes_df = telesat_passes_df.head(N_passes)

lines = [None] * N_passes
for n, pass_df in passes_df.iterrows():
    line, = ax.plot(pass_df.ttof, pass_df.rrange / 1000., '--', linewidth=0.5, c=[0.6]*3, zorder=1)
    lines[n] = line

ax.set_xlabel('time of flight [s]')
ax.set_ylabel('range [km]')
ax.grid()
ax.set_xlim((0, telesat_passes_df.iloc[-1].StopTof))

a = 2
b = 3
both = res.X[a,:-1] * res.X[b,:-1] == 1
only_a = (res.X[a,:-1] - res.X[b,:-1]) > 0
only_b = (res.X[b,:-1] - res.X[a,:-1]) > 0 

contact_time_a, overlap_a, power_consumption_a, total_link_time_a, isend_dB_list_a = res.problem.evaluate_passes(res.X[a,:])
contact_time_b, overlap_b, power_consumption_b, total_link_time_b, isend_dB_list_b = res.problem.evaluate_passes(res.X[b,:])

for j in np.where(both)[0]:
    lines[j].set_color('k')
    lines[j].set_linestyle('-')
    lines[j].set_zorder(2)
    
for j in np.where(only_a)[0]:
    lines[j].set_color('r')
    lines[j].set_linestyle('-')
    lines[j].set_zorder(2)

for j in np.where(only_b)[0]:
    lines[j].set_color('b')
    lines[j].set_linestyle('-')
    lines[j].set_zorder(2)

apply_report_formatting()
plt.tight_layout()
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_compare.svg')
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_compare.pgf')
plt.close()

In [ ]:
print("Contact: a=%.1f s, b=%.1f s" % (contact_time_a, contact_time_b))
print("Power: a=%d J, b=%d J" % (power_consumption_a, power_consumption_b))
print("Link: a=%.1f s, b=%.1f s" % (total_link_time_a, total_link_time_b))

fig = plt.figure(figsize=(3.2, 2.4))
ax = fig.add_subplot(1,1,1)

lines = [None] * N_passes
for n, pass_df in passes_df.loc[res.X[a,:-1]].iterrows():
    line, = plt.plot(pass_df.ttof, isend_dB_list_a[n], 'r-', linewidth=0.5)
    lines[n] = line

ax.set_ylabel("$E_b / N_0 [J]$")
ax.set_xlabel("Time of flight [s]")

plt.hlines(isend_req, 0, 10000, linestyles='--', linewidth=0.5)
apply_report_formatting()
plt.tight_layout()
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_a.svg')
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_a.pgf')
plt.close()

fig = plt.figure(figsize=(3.2, 2.4))

lines = [None] * N_passes
for n, pass_df in passes_df.loc[res.X[b,:-1]].iterrows():
    line, = plt.plot(pass_df.ttof, isend_dB_list_b[n], 'b-', linewidth=0.5)
    lines[n] = line

ax.set_ylabel("$E_b / N_0 [J]$")
ax.set_xlabel("Time of flight [s]")

plt.hlines(isend_req, 0, 10000, linestyles='--', linewidth=0.5)
apply_report_formatting()
plt.tight_layout()
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_b.svg')
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_b.pgf')
plt.close()